# Introduction to the dataset

Import libreries and load data

In [6]:
import pickle
from qiskit.visualization import dag_drawer

In [3]:
# Define the file path
file_path = 'random_circuits/basis_rotations+cx_qubits_5_gates_0-19.pkl'

# Open the file in binary read mode and load the data
with open(file_path, 'rb') as file:
    data = pickle.load(file)

Introduction to the dataset

In [5]:
print(len(data), len(data[0]), len(data[0][0]))
dag_drawer(data[0][0]["dag"], filename= "random_circuits/images/dag.png")

10000 2 109
